In [13]:
import requests
import pandas as pd
import plotly.express as px

from dotenv import load_dotenv   #for python-dotenv method
load_dotenv()                    #for python-dotenv method

import os 

liste_villes = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg",
"Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon",
"Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer",
"Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]


COORDONNEES D'UNE SEULE VILLE

In [5]:
# ESSAI MANUEL POUR UNE SEULE VILLE 

url = "https://nominatim.openstreetmap.org/search?"
headers = {"accept": "application/json", 'user_agent':'charles.mittempergher@gmail.com'}
p = {'city' : 'Toulouse', 'country':'France', 'format':'json', 'limit':1}

response = requests.get(url, headers=headers, params=p)

BOUCLE POUR LES COORDONNEES TOUTES LES VILLES

In [ ]:
# RECUPERATION DES DONNEES POUR TOUTE LA LISTE DES VILLES
url = "https://nominatim.openstreetmap.org/search?"
headers = {'user_agent':'charles.mittempergher@gmail.com'}

df_villes = pd.DataFrame(columns=['ville','lat','lon'])

for i in range(len(liste_villes)):
    
    p = {'city' : {liste_villes[i]}, 'country':'France', 'format':'json', 'limit':1, 'email':'charles@gmail.com'}

    r = requests.get(url, headers=headers, params=p)
    ville = r.json()[0]['name']
    lat   = r.json()[0]['lat']
    lon   = r.json()[0]['lon']

    df_villes.loc[i] = [ville, lat,lon]
    print(f'Ajout de {liste_villes[i]} effectué')

In [8]:
df_villes.head()

,ville,lat,lon
0,Mont Saint-Michel,48.6359541,-1.511459954959514
1,Saint-Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


 DEBUT DE LA PARTIE AVEC OPENWEATHER

In [14]:
url_ow = "https://api.openweathermap.org/data/2.5/forecast?"

lat = 43.2961743
lon = 5.3699525
p = {'lat' : lat, 'lon' : lon, 'units':'metric', 'appid' : os.environ.get('openweather_active')}
response = requests.get(url_ow, params=p)


In [15]:
response

<Response [200]>

In [10]:
response.json().get('list')

CREATION DU DATAFRAME AVEC LES PREVISIONS METEO POUR UNE SEULE VILLE

In [16]:
liste_previsions_une_ville = response.json().get('list')

len(liste_previsions_une_ville)
df_prevision_une_ville = pd.DataFrame(columns=['Ville','date','heure', 'temp','id_type_temps','type_temps'])
for i in range(len(liste_previsions_une_ville)):
    
    ville = 'Marseille'
    dt_text= liste_previsions_une_ville[i]['dt_txt']
    date = dt_text.split(" ")[0]
    heure= dt_text.split(" ")[1]
    temp=liste_previsions_une_ville[i]['main']['temp']
    id_type_temps=liste_previsions_une_ville[i]['weather'][0]['id']
    type_temps=liste_previsions_une_ville[i]['weather'][0]['main']

    df_prevision_une_ville.loc[i] = [ville,date,heure,temp,id_type_temps,type_temps]

In [ ]:
df_prevision_une_ville.head()

,Ville,date,heure,temp,id_type_temps,type_temps
0,Marseille,2024-11-14,18:00:00,12.29,800,Clear
1,Marseille,2024-11-14,21:00:00,12.83,800,Clear
2,Marseille,2024-11-15,00:00:00,13.10,800,Clear
3,Marseille,2024-11-15,03:00:00,13.00,800,Clear
4,Marseille,2024-11-15,06:00:00,12.73,800,Clear


ON PREND PAS LES PREVISIONS ENTRE MINUIT ET 6H

In [ ]:
mask = df_prevision_une_ville['heure'] >= '06:00:00'
data = df_prevision_une_ville[mask]

In [ ]:
data.head()

,Ville,date,heure,temp,id_type_temps,type_temps
0,Marseille,2024-11-14,18:00:00,12.29,800,Clear
1,Marseille,2024-11-14,21:00:00,12.83,800,Clear
4,Marseille,2024-11-15,06:00:00,12.73,800,Clear
5,Marseille,2024-11-15,09:00:00,13.82,800,Clear
6,Marseille,2024-11-15,12:00:00,14.84,800,Clear


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PASSAGE AUX CHOSES SERIEUSES

BOUCLE POUR TOUTES LES PREVISIONS DE TOUTES LES VILLES

In [17]:
url_ow = "https://api.openweathermap.org/data/2.5/forecast?"

df_prevision_toutes_les_villes = pd.DataFrame(columns=['Ville','date','heure','Part_of_day', 'temp','id_type_temps','type_temps','couv_nuageuse','vent','prob_pluie','pluie'])

# BOUCLE SUR LES VILLES 
for v in range(len(df_villes)):

    lat = df_villes.loc[v,"lat"]
    lon = df_villes.loc[v,"lon"]
    p = {'lat' : lat, 'lon' : lon, 'units':'metric', 'appid' :  os.environ.get('openweather_active')}
    response = requests.get(url_ow, params=p)
    response.json().get('list')

    liste_previsions_une_ville_final = response.json().get('list')
    nb_prev = len(liste_previsions_une_ville_final)
    # BOUCLES SUR LES PREVISIONS
    for i in range(len(liste_previsions_une_ville_final)):
        
        ville = df_villes.loc[v,"ville"]
        dt_text= liste_previsions_une_ville_final[i]['dt_txt']
        date = dt_text.split(" ")[0]
        heure= dt_text.split(" ")[1]
        temp=liste_previsions_une_ville_final[i]['main']['temp']
        id_type_temps=liste_previsions_une_ville_final[i]['weather'][0]['id']
        type_temps=liste_previsions_une_ville_final[i]['weather'][0]['main']
        couv_nuageuse=liste_previsions_une_ville_final[i]['clouds']['all']
        vent=liste_previsions_une_ville_final[i]['wind']['speed']
        prob_pluie=liste_previsions_une_ville_final[i]['pop']
        pluie = 0
        if 'rain' in liste_previsions_une_ville_final[i].keys():
            pluie = liste_previsions_une_ville_final[i]['rain']['3h']
        pod = liste_previsions_une_ville_final[i]['sys']['pod']

        df_prevision_toutes_les_villes.loc[len(df_prevision_toutes_les_villes)+1] = [ville,date,heure,pod,temp,id_type_temps,type_temps,couv_nuageuse,vent,prob_pluie,pluie]
        

In [18]:
mask = df_prevision_toutes_les_villes['heure'] >= '06:00:00'
data_toutes_les_villes = df_prevision_toutes_les_villes[mask]

ON REGARDE LES DIFFERENTES VALEURS DU TYPE DE METEO POUR ETABLIR UN CLASSEMENT

In [19]:
data_toutes_les_villes['id_type_temps'].unique()

array([802, 803, 801, 804, 500, 501, 800, 601, 600, 502], dtype=int64)

MAPPING ET CREATION D'UNE NOUVELLE COLONNE AVEC LA NOTE DE LA PREVISION

In [20]:
classement = {800: 10, 801:9,802:8,803:7,804:6,500:4,501:3, 601:2}
data_toutes_les_villes['note_type_temps']=data_toutes_les_villes['id_type_temps'].apply(lambda x: classement[x] if x in classement else 1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_21084\3510448894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_toutes_les_villes['note_type_temps']=data_toutes_les_villes['id_type_temps'].apply(lambda x: classement[x] if x in classement else 1)


In [21]:
data_toutes_les_villes.head()

,Ville,date,heure,Part_of_day,temp,id_type_temps,type_temps,couv_nuageuse,vent,prob_pluie,pluie,note_type_temps
1,Mont Saint-Michel,2024-11-15,18:00:00,n,9.92,802,Clouds,47,2.26,0.0,0.0,8
2,Mont Saint-Michel,2024-11-15,21:00:00,n,8.08,803,Clouds,80,2.12,0.0,0.0,7
5,Mont Saint-Michel,2024-11-16,06:00:00,n,4.40,801,Clouds,11,1.17,0.0,0.0,9
6,Mont Saint-Michel,2024-11-16,09:00:00,d,6.48,802,Clouds,30,0.83,0.0,0.0,8
7,Mont Saint-Michel,2024-11-16,12:00:00,d,9.97,802,Clouds,38,1.20,0.0,0.0,8


In [22]:
total_par_ville = data_toutes_les_villes.groupby('Ville')['note_type_temps'].sum()

In [23]:
total_par_ville

Ville
Aigues-Mortes                  199
Aix-en-Provence                222
Amiens                         180
Annecy                         191
Avignon                        208
Bayeux                         175
Bayonne                        190
Besançon                       167
Biarritz                       189
Bormes-les-Mimosas             241
Carcassonne                    206
Cassis                         224
Château du Haut-Kœnigsbourg    155
Collioure                      228
Colmar                         160
Dijon                          174
Eguisheim                      158
Gorges du Verdon               221
Grenoble                       195
La Rochelle                    179
Le Havre                       169
Lille                          158
Lyon                           200
Marseille                      220
Mont Saint-Michel              169
Montauban                      203
Nîmes                          209
Paris                          176
Rouen         

In [24]:
moyenne_temp_par_ville = data_toutes_les_villes.groupby('Ville')['temp'].mean()

ON FUSIONNE LES TROIS TABLES POUR AVOIR NOS DONNEES FINALES (VILLES ET COORDONNEES, SCORE, TEMPERATURE MOYENNE)

In [25]:
df_final = df_villes.merge(total_par_ville,left_on='ville',right_on='Ville')
df_final = df_final.merge(moyenne_temp_par_ville,left_on='ville', right_on='Ville')

CHANGEMENT DU TYPE DE DONNEES POUR LATITUDE ET LONGITUDE

In [26]:
df_final['lat'] = df_final['lat'].astype('float64')
df_final['lon'] = df_final['lon'].astype('float64')

AFFICHAGE DU DATASET FINAL UTILISE POUR LA CARTE

In [27]:
df_final.sort_values("note_type_temps", ascending=False)

,ville,lat,lon,note_type_temps,temp
18,Bormes-les-Mimosas,43.150697,6.341928,241,13.463333
27,Collioure,42.525050,3.083155,228,14.706000
19,Cassis,43.214036,5.539632,224,14.137000
21,Aix-en-Provence,43.529842,5.447474,222,12.424000
17,Gorges du Verdon,43.749656,6.328562,221,6.619333
20,Marseille,43.296174,5.369953,220,14.945667
29,Tarascon-sur-Ariège,42.845802,1.606759,210,12.212667
24,Nîmes,43.837425,4.360069,209,11.902000
22,Avignon,43.949249,4.805901,208,11.828333
30,Toulouse,43.604462,1.444247,207,12.013667


CREATION DE LA CARTE

In [28]:
fig = px.scatter_mapbox(df_final,lat="lat", lon="lon", color="note_type_temps", hover_name="ville", hover_data='temp', height=700,
mapbox_style="open-street-map",title='Où on part ce weekend ?',zoom=4.5,color_continuous_scale=["red","orange","yellowgreen","green"], size="temp", center={'lon':2.232999,"lat":46.327408})
fig.update_layout(title_x=0.5)
fig.show()